In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col

In [2]:
# Create a Spark session
spark = SparkSession.builder.appName("SQL tasks").getOrCreate()
# load data

**Task 2.0**

In [3]:
CustomerDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Customer.csv")
PurchaseDF=spark.read.option("header",True).option("inferSchema",True).csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/Purchase.csv")
PurchaseDF.printSchema()
CustomerDF.printSchema()
PurchaseDF.cache()
CustomerDF.cache()

root
 |-- TransID: integer (nullable = true)
 |-- CustID: integer (nullable = true)
 |-- TransTotal: double (nullable = true)
 |-- TransNumItems: integer (nullable = true)
 |-- TransDesc: string (nullable = true)

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- CountryCode: integer (nullable = true)
 |-- Salary: double (nullable = true)



DataFrame[ID: int, Name: string, Age: int, CountryCode: int, Salary: double]

**Task 2.1**

In [4]:
PurchaseDF.createOrReplaceTempView("PurchaseView")
T1=spark.sql("""SELECT * FROM PurchaseView P WHERE TransTotal<=600""")
T1.printSchema()
T1.write.option("header",True).mode("overwrite").csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/T1.csv")
T1.show(10)

root
 |-- TransID: integer (nullable = true)
 |-- CustID: integer (nullable = true)
 |-- TransTotal: double (nullable = true)
 |-- TransNumItems: integer (nullable = true)
 |-- TransDesc: string (nullable = true)

+-------+------+------------------+-------------+--------------------+
|TransID|CustID|        TransTotal|TransNumItems|           TransDesc|
+-------+------+------------------+-------------+--------------------+
|      7|  2681|487.39927274738744|            2|Grocery: Rice Flo...|
|     10|  5306| 582.7577969199737|           14|Pet: Food Toy Bed...|
|     12| 43639| 471.4799539815962|           12|Beauty: Skincare ...|
|     20| 22546| 436.4924591172179|            8|Beauty: Cream Bod...|
|     24| 33718|461.38901609618176|            5|Pet: Food Toy Bed...|
|     25| 23982| 401.3531312605189|            5|Clothes: Jumper S...|
|     27| 33566| 388.0662792207821|           10|Grocery: Egg Brea...|
|     29| 30481| 420.8616091990247|            8|Other: Electronic...|
|    

**Task 2.2**

In [5]:
PurchaseDF.createOrReplaceTempView("PurchaseView")
T1=spark.sql("""SELECT * FROM PurchaseView P WHERE TransTotal<=600""")
T1.createOrReplaceTempView("T1View")
T2=spark.sql("""SELECT TransNumItems,round(PERCENTILE_APPROX(TransTotal,0.5),2) AS MEDIANTOTAL, round(MIN(TransTotal),2) AS MINTOTAL, round(MAX(TransTotal),2) AS MAXTOTAL FROM T1View GROUP BY TransNumItems""")
T2.show(10)
T2.write.option("header",True).mode("overwrite").csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/T2.csv")

+-------------+-----------+--------+--------+
|TransNumItems|MEDIANTOTAL|MINTOTAL|MAXTOTAL|
+-------------+-----------+--------+--------+
|           12|     304.26|    10.0|  599.99|
|            1|     304.45|   10.02|   600.0|
|           13|     305.78|   10.01|  599.99|
|            6|     304.73|    10.0|  599.99|
|            3|     306.15|   10.01|  599.99|
|            5|     306.76|   10.01|  599.99|
|           15|     305.32|   10.01|  599.99|
|            9|     305.02|   10.01|  599.99|
|            4|     305.48|   10.01|   600.0|
|            8|      305.3|    10.0|   600.0|
+-------------+-----------+--------+--------+
only showing top 10 rows



**Task 2.3**

In [6]:
PurchaseDF.createOrReplaceTempView("PurchaseView")
T1=spark.sql("""SELECT * FROM PurchaseView P WHERE TransTotal<=600""")
T1.createOrReplaceTempView("T1View")
CustomerDF.where("Age between 18 and 25").createOrReplaceTempView("YoungCustomerView")
T3=spark.sql("""SELECT cus.ID, cus.AGE, round(SUM(TransTotal),2) AS TotalAmount,SUM(TransNumItems) AS TotalNumberOfItems FROM T1View t1 INNER JOIN YoungCustomerView cus ON t1.CustID=cus.ID GROUP BY cus.ID, cus.AGE""")
T3.show(10)
T3.write.option("header",True).mode("overwrite").csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/T3.csv")

+-----+---+-----------+------------------+
|   ID|AGE|TotalAmount|TotalNumberOfItems|
+-----+---+-----------+------------------+
| 4111| 22|    8244.76|               261|
|41048| 18|    6984.26|               218|
|31183| 19|    9896.91|               258|
|34623| 18|    8941.63|               261|
|47510| 21|    8770.89|               201|
|27451| 25|   10666.55|               296|
|39653| 23|   11394.68|               314|
|38570| 18|   11423.18|               266|
|22855| 19|    8890.04|               268|
|40979| 25|    7811.62|               230|
+-----+---+-----------+------------------+
only showing top 10 rows



**Task 2.4**

In [7]:
PurchaseDF.createOrReplaceTempView("PurchaseView")
T1=spark.sql("""SELECT * FROM PurchaseView P WHERE TransTotal<=600""")
T1.createOrReplaceTempView("T1View")
#CustomerDF.where("Age between 18 and 25").createOrReplaceTempView("YoungCustomerView")
CustomerDF.createOrReplaceTempView("CustomerView")
YoungCustomer=spark.sql("""SELECT * FROM CustomerView C WHERE Age between 18 and 25""")
YoungCustomer.createOrReplaceTempView("YoungCustomerView")
T3=spark.sql("""SELECT cus.ID, cus.AGE, round(SUM(TransTotal),2) AS TotalAmount,SUM(TransNumItems) AS TotalNumberOfItems FROM T1View t1 INNER JOIN YoungCustomerView cus ON t1.CustID=cus.ID GROUP BY cus.ID, cus.AGE""")
T3.createOrReplaceTempView("T3View")
T4=spark.sql("""SELECT c1.ID as C1_ID, c2.ID as C2_ID, c1.Age as Age1, c2.age as Age2, round(c1.TotalAmount,2) as TotalAmount1, round(c2.TotalAmount,2) TotalAmount2, c1.TotalNumberOfItems  as TotalNumberOfItems1, c2.TotalNumberOfItems as TotalNumberOfItems2 FROM T3View c1 inner join T3View c2 on c1.Age<c2.Age where c1.TotalAmount>c2.TotalAmount and c1.TotalNumberOfItems<c2.TotalNumberOfItems""")
T4.show(10)
T4.write.option("header",True).mode("overwrite").csv("C:/Users/ganer/Documents/classes2023/fall/Big_data/project3/T4.csv")


+-----+-----+----+----+------------+------------+-------------------+-------------------+
|C1_ID|C2_ID|Age1|Age2|TotalAmount1|TotalAmount2|TotalNumberOfItems1|TotalNumberOfItems2|
+-----+-----+----+----+------------+------------+-------------------+-------------------+
|31183| 4111|  19|  22|     9896.91|     8244.76|                258|                261|
|47510| 4111|  21|  22|     8770.89|     8244.76|                201|                261|
| 5031| 4111|  21|  22|      8280.8|     8244.76|                189|                261|
| 1333| 4111|  18|  22|     8837.36|     8244.76|                210|                261|
|27656| 4111|  19|  22|     8706.59|     8244.76|                198|                261|
| 2646| 4111|  19|  22|     9261.67|     8244.76|                222|                261|
|25045| 4111|  21|  22|     8529.23|     8244.76|                237|                261|
|18473| 4111|  21|  22|     8496.62|     8244.76|                238|                261|
|34899| 41

In [8]:
spark.sparkContext.stop()